In [1]:
import numpy as np

def svd_decompose(A):
    """
    A = U @ np.diag(S) @ VT 으로 분해
    full_matrices=False 로 하면 U.shape=(m,r), VT.shape=(r,n) 이 됨 (r = rank estimate)
    """
    U, S, VT = np.linalg.svd(A, full_matrices=False)
    return U, S, VT

def rank1_matrices(U, S, VT):
    """
    각 i에 대해 A_i = sigma_i * (u_i outer v_i) 리스트 반환
    """
    m, r = U.shape
    r, n = VT.shape
    return [S[i] * np.outer(U[:, i], VT[i, :]) for i in range(r)]

def rank_k_approx(U, S, VT, k):
    """
    앞 k개 성분만 더해 랭크-k 근사 A_k 반환
    """
    return sum(S[i] * np.outer(U[:, i], VT[i, :]) for i in range(k))

def spectral_norm(A):
    """
    스펙트럴 노름 = 최대 특이값
    """
    return np.linalg.svd(A, compute_uv=False)[0]

def approximation_error(A, A_k):
    """
    스펙트럴 노름 기준 근사 오차
    """
    return spectral_norm(A - A_k)

def storage_cost(m, n, k):
    """
    원본: m*n
    랭크-k 근사: k*(m + n)
    """
    return {'original': m * n, 'approx_k': k * (m + n)}

# ───────── 예제 실행 ─────────
if __name__ == "__main__":
    # (1) 예제 행렬 생성 (임의의 작은 이미지나 random matrix)
    m, n = 100, 150
    A = np.random.rand(m, n)

    # (2) SVD
    U, S, VT = svd_decompose(A)
    r = len(S)  # 랭크 추정치

    # (3) 각 랭크-1 행렬
    A_list = rank1_matrices(U, S, VT)
    print(f"랭크-1 행렬 개수 (non-zero sigma): {r}")
    print(f"첫 세 특이값: {S[:3]}")

    # (4) 랭크-k 근사 & 오차 & 저장비교
    for k in [1, 2, 5, 10]:
        A_k = rank_k_approx(U, S, VT, k)
        err = approximation_error(A, A_k)
        cost = storage_cost(m, n, k)
        print(f"\n— k={k} —")
        print(f"근사 오차 ‖A - A_k‖₂ = {err:.4e}  (이론값 σₖ₊₁ = {S[k]:.4e})")
        print(f"저장 비용: 원본={cost['original']}  vs 근사={cost['approx_k']}")

    # (5) 스펙트럴 노름 (전체)
    print(f"\n스펙트럴 노름 ‖A‖₂ = {spectral_norm(A):.4e} (= σ₁)")


랭크-1 행렬 개수 (non-zero sigma): 100
첫 세 특이값: [61.39508097  6.15452311  6.01029253]

— k=1 —
근사 오차 ‖A - A_k‖₂ = 6.1545e+00  (이론값 σₖ₊₁ = 6.1545e+00)
저장 비용: 원본=15000  vs 근사=250

— k=2 —
근사 오차 ‖A - A_k‖₂ = 6.0103e+00  (이론값 σₖ₊₁ = 6.0103e+00)
저장 비용: 원본=15000  vs 근사=500

— k=5 —
근사 오차 ‖A - A_k‖₂ = 5.7441e+00  (이론값 σₖ₊₁ = 5.7441e+00)
저장 비용: 원본=15000  vs 근사=1250

— k=10 —
근사 오차 ‖A - A_k‖₂ = 5.3838e+00  (이론값 σₖ₊₁ = 5.3838e+00)
저장 비용: 원본=15000  vs 근사=2500

스펙트럴 노름 ‖A‖₂ = 6.1395e+01 (= σ₁)
